<a href="https://colab.research.google.com/github/wanjeans33/Deep-Learning/blob/main/Day1_Task_Perceptron_Titanic_handout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

We download the data:

In [ ]:
! wget https://gitlab.com/andras.simonyi/10_days_AI_training_data/raw/65de5908dccf120762b305238e02610a8c18a3f9/titanic_train.csv

--2024-09-02 12:36:41--  https://gitlab.com/andras.simonyi/10_days_AI_training_data/raw/65de5908dccf120762b305238e02610a8c18a3f9/titanic_train.csv
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61194 (60K) [text/plain]
Saving to: ‘titanic_train.csv’

titanic_train.csv   100%[===================>]  59.76K  --.-KB/s    in 0.002s  

2024-09-02 12:36:41 (23.4 MB/s) - ‘titanic_train.csv’ saved [61194/61194]



# A perceptron implementation in NumPy

We create a Perceptron class, which mimics scikit-learn estimators by providing a `fit` and a `predict` method:
+ The `predict` method returns a vector of predictions for an array of samples, while
+ the `fit` method initializes the model parameters and trains the model using the perceptron learning rule.

In [ ]:
import numpy as np
from numpy.random import permutation, seed
seed(12) # Fix the random seed

class Perceptron:
    """A simple implementation of the classic single-neuron perceptron model.

    Attrs:
        fitted (bool): Whether the model has been fitted.
        n_features (int): Number of input features.
        weights (numpy array): The model's weights.
        bias (float): The model's bias.
    """

    def __init__(self):
        """Create a perceptron model.

        Returns:
            A new perceptron instance.
        """
        self.fitted = False

    def fit(self, X, y, n_epochs=10, lr=0.01):
        """Fit the model to a data set.

        Args:
            X (numpy array of shape (n_samples, n_features)): Training data.
            y (numpy array of shape (n_samples,)): Target binary labels.
            n_epochs (int): Number of training epochs.
            lr (float): Learning rate.
        """
        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y, dtype=np.int32)
        n_samples, self.n_features = X.shape

        # Initialization
        # As we know, tha ONLY neural model that can be initialized to zeroes
        # is the Perceptron, if we use the perceptron learning rule.
        # Please initialize the weights and biases to zero!
        # Use Numpy!
        # Bear in mind, that the weights are a VECTOR, not a one dimensional matrix!
        # Bias is a scalar.
        self.weights = np.zeros(self.n_features) * 0.0
        self.bias = 0.0

        # Training
        # Implement the main loop, use the epoch parameter!
        for e in range(n_epochs):
            print("Starting epoch", e)

            # Random shuffle - in  a tricky way.
            # Generate a permutation mask with numpy
            # which we will use to index into the data, thus realizing "shuffling".
            # Numpy has a permutation function.
            # Please bear in mind, how many datapoints do we have.
            # We have a variable for it above...
            perm = np.random.permutation(n_samples)

            # Do the epoch!
            # Observe the trick, please!
            # We use the permutation mask to index into X and y,
            # and we generate and iterator of tuples.
            # Then we parse the intividual tuples (x,y pairs), and loot through them.
            for x, label in zip(X[perm], y[perm]): #: you can uncomment this, if you have understood.
                # Please convert the 1/0 label to 1/-1 label
                # 1 should remain 1, 0 should become -1
                # Use some easy mathematics, or whatever you feel like.
                y_ = 2 * label -1

                # Implement the update rule!
                # If the simple product of the output of our neuron and the true label is
                # less than or equal to zero (negative), we will update.
                # Remember, the activation is W times x plus the bias.
                # Please bear in mind, that the "times" here is denoting the dot product!
                if y_ * (np.dot(x,self.weights)+self.bias)<=0:
                    # So, we have an error. We should "update" the weights and the bias!
                    # We just add to the weight the multiplication of the input vector
                    # and the true label, mutiplied by the learning rate
                    # (we have a parameter for that).
                    self.weights += lr * y_ * x
                    # For the bias we have the "virtual input" of 1, so we only add true label
                    # multiplied by the learning rate.
                    self.bias += lr * y_

                # Lo and behold, THAT'S IT!
                # We have implemented a perceptron!

        self.fitted = True
        print("Finished training.")

    def predict(self, X):
        """Predict labels for samples.

        Args:
            X (numpy array of shape (n_samples, n_features)):  Samples.

        Returns:
            A numpy array of shape (n_samples,) containing the predicted labels.
        """
        if not self.fitted:
            raise ValueError("Perceptron model is not fitted")
        elif X.shape[1] != self.n_features:
            raise ValueError(f"Incorrect number of input features (expected {self.n_features})")
        else:

            # Please think through and explain the trainer, WHY this is a matmul!?
            activations = np.matmul(X, self.weights) + self.bias
            signs = np.sign(activations).astype(int)
            # Tricky conversion of signs to 0/1 labels
            return (signs + signs * signs) // 2

# Trying it out -- on the Titanic data set

Short description of the data set:

> The titanic [...] data frames describe the survival status of individual passengers on the Titanic.

> Non-obvious variables:

>- Pclass -- Passenger Class  (1 = 1st; 2 = 2nd; 3 = 3rd)
>- Survived -- Survival  (0 = No; 1 = Yes)
>- SibSp -- Number of Siblings/Spouses Aboard
>- Parch -- Number of Parents/Children Aboard
>- Fare (ticket price in British pound)
>- Embarked -- Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


df = pd.read_csv("titanic_train.csv")

For the sake of simplicity, we divide our data only into a training and a validation part:

In [ ]:
df_train, df_valid = train_test_split(df, test_size=0.1)
df_train.reset_index(inplace=True)
df_valid.reset_index(inplace=True)
print("train shape:", df_train.shape)
print("validation shape:", df_valid.shape)

train shape: (801, 13)
validation shape: (90, 13)


## Inspecting and cleaning the data

In [ ]:
df_train.head()

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,715,716,0,3,"Soholt, Mr. Peter Andreas Lauritz Andersen",male,19.0,0,0,348124,7.6500,F G73,S
1,319,320,1,1,"Spedden, Mrs. Frederic Oakley (Margaretta Corn...",female,40.0,1,1,16966,134.5000,E34,C
2,829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0000,B28,NaN
3,79,80,1,3,"Dowdell, Miss. Elizabeth",female,30.0,0,0,364516,12.4750,NaN,S
4,484,485,1,1,"Bishop, Mr. Dickinson H",male,25.0,1,0,11967,91.0792,B49,C


In [ ]:
df_train.describe(include="all")

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,801.000000,801.000000,801.000000,801.000000,801,801,642.000000,801.000000,801.000000,801,801.000000,179,799
unique,NaN,NaN,NaN,NaN,801,2,NaN,NaN,NaN,627,NaN,133,3
top,NaN,NaN,NaN,NaN,"Lemberopolous, Mr. Peter L",male,NaN,NaN,NaN,1601,NaN,G6,S
freq,NaN,NaN,NaN,NaN,1,517,NaN,NaN,NaN,7,NaN,4,578
mean,439.566792,440.566792,0.383271,2.309613,NaN,NaN,29.634097,0.518102,0.373283,NaN,32.386844,NaN,NaN
std,256.901656,256.901656,0.486487,0.834578,NaN,NaN,14.516042,1.120261,0.786915,NaN,50.556727,NaN,NaN
min,0.000000,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,216.000000,217.000000,0.000000,2.000000,NaN,NaN,20.000000,0.000000,0.000000,NaN,7.895800,NaN,NaN
50%,441.000000,442.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.400000,NaN,NaN
75%,659.000000,660.000000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [ ]:
df_train.isna().sum()

,0
index,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,159
SibSp,0
Parch,0
Ticket,0


Based on our inspection, we drop the Cabin, Ticket, PassengerId, Name and index columns, since they are unusable for the prediction task:

In [ ]:
columns_to_drop = ["Cabin", "Ticket", "PassengerId", "Name", "index"]
df_train = df_train.drop(columns=columns_to_drop)
df_valid = df_valid.drop(columns=columns_to_drop)
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,19.0,0,0,7.6500,S
1,1,1,female,40.0,1,1,134.5000,C
2,1,1,female,62.0,0,0,80.0000,NaN
3,1,3,female,30.0,0,0,12.4750,S
4,1,1,male,25.0,1,0,91.0792,C


We encode the the gender of passenger by numbers, and, as a primitive form of data imputation, replace missing age values with the mean age in the training data:

In [ ]:
age_mean = df_train.Age.mean()
replacements = {"Sex": {"male": 1, "female":0}, "Age": {np.NaN: age_mean}}
df_train.replace(replacements, inplace=True)
df_valid.replace(replacements, inplace=True)
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,19.0,0,0,7.6500,S
1,1,1,0,40.0,1,1,134.5000,C
2,1,1,0,62.0,0,0,80.0000,NaN
3,1,3,0,30.0,0,0,12.4750,S
4,1,1,1,25.0,1,0,91.0792,C


We drop the remaining rows with missing data:

In [ ]:
print("Train and test length before dropping:", len(df_train), len(df_valid))
df_train.dropna(inplace=True)
df_train.reset_index()
df_valid.dropna(inplace=True)
df_valid.reset_index()
print("Train and test length after dropping:", len(df_train), len(df_valid))

Train and test length before dropping: 801 90
Train and test length after dropping: 799 90


Finally, we one-hot encode the Embarked column.

In [ ]:
df_train = pd.get_dummies(df_train)
df_valid = pd.get_dummies(df_valid)
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,19.0,0,0,7.6500,False,False,True
1,1,1,0,40.0,1,1,134.5000,True,False,False
3,1,3,0,30.0,0,0,12.4750,False,False,True
4,1,1,1,25.0,1,0,91.0792,True,False,False
5,0,3,1,70.5,0,0,7.7500,False,True,False


## Fitting a perceptron

In [ ]:
p = Perceptron()

input_cols = list(df_train.columns.values)[1:]

X_train = df_train[input_cols].to_numpy()
y_train = df_train.Survived.to_numpy()

p.fit(X_train, y_train, n_epochs=100, lr=0.001)

Starting epoch 0
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
Starting epoch 20
Starting epoch 21
Starting epoch 22
Starting epoch 23
Starting epoch 24
Starting epoch 25
Starting epoch 26
Starting epoch 27
Starting epoch 28
Starting epoch 29
Starting epoch 30
Starting epoch 31
Starting epoch 32
Starting epoch 33
Starting epoch 34
Starting epoch 35
Starting epoch 36
Starting epoch 37
Starting epoch 38
Starting epoch 39
Starting epoch 40
Starting epoch 41
Starting epoch 42
Starting epoch 43
Starting epoch 44
Starting epoch 45
Starting epoch 46
Starting epoch 47
Starting epoch 48
Starting epoch 49
Starting epoch 50
Starting epoch 51
Starting epoch 52
Starting epoch 53
Starting epoch 54
Starting epoch 55
St

In [ ]:
y_train_predicted = p.predict(X_train)

X_valid = df_valid[input_cols].to_numpy()
y_valid = df_valid.Survived.to_numpy()

y_valid_predicted = p.predict(X_valid)

Let's see the metrics on our training and validation data:

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_train, y_train_predicted, labels=[1], target_names=["Survivor"]))

              precision    recall  f1-score   support

    Survivor       0.72      0.74      0.73       305

   micro avg       0.72      0.74      0.73       305
   macro avg       0.72      0.74      0.73       305
weighted avg       0.72      0.74      0.73       305



In [ ]:
print(classification_report(y_valid, y_valid_predicted, labels=[1], target_names=["Survivor"]))

              precision    recall  f1-score   support

    Survivor       0.72      0.60      0.66        35

   micro avg       0.72      0.60      0.66        35
   macro avg       0.72      0.60      0.66        35
weighted avg       0.72      0.60      0.66        35

